In [25]:
from pymongo import MongoClient

host = "localhost"
port = 27017
client = MongoClient(f"mongodb://{host}:{port}/")
db = client["LLMQueryAgent"]
print(db.list_collection_names())
collection = db["Functional"]

['Functional']


In [75]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are an expert AI assistant specializing in generating efficient and accurate MongoDB queries. 
The database schema is: {schema} 

When returning a query, output only the query itself—no extra words or explanations."""),
        MessagesPlaceholder(variable_name="history"),
        ("user", "{query}")
    ]
)

prompt.invoke(
    {
        "schema": collection.find_one({}),
        "history": [
            {"role": "user", "content": ""},
            {"role": "assistant", "content": ""}
        ],
        "query": "What is the capital of France?"
    }
)


ChatPromptValue(messages=[SystemMessage(content="You are an expert AI assistant specializing in generating efficient and accurate MongoDB queries. \nThe database schema is: {'_id': ObjectId('67ced2ca97c2a94c1b06d3d1'), 'frame_time': datetime.datetime(2024, 11, 4, 12, 5, 5, 428000), 'frame_time_epoch': 1730721905428675000, 'frame_number': '1496', 'protocol': 'HTTP2/JSON', 'src': 'UDR', 'dst': 'UDM', 'tcp_srcport': '2760', 'tcp_dstport': '34332', 'udp_srcport': '', 'udp_dstport': '', 'sctp_srcport': '', 'sctp_dstport': '', 'info': 'HEADERS[21]: /nudr-dr/v2/subscription-data/imsi-912116000000001/context-data/smf-registrations, 404', 'Error_Markers': {'type': 'http2', 'status': '404', 'request': '/nudr-dr/v2/subscription-data/imsi-912116000000001/context-data/smf-registrations', 'method': 'get'}, 'Message_Identifier': {'message': 'HEADERS[21]: /nudr-dr/v2/subscription-data/imsi-912116000000001/context-data/smf-registrations, 404'}, 'job_id': '98ad489a-9158-44f8-8f04-7b7e362e0d74'} \n\nWhen

In [10]:
from langchain_openai import ChatOpenAI

# llama-server -m qwen2.5-7b-instruct-q4_0.gguf \
#     --threads -1 --host 0.0.0.0 --port 8000 \
#     --log-file chat_llm \
#     --ctx-size 8192 \
#     --n-predict 512 \
#     --temp 0.5 \
#     --top-k 10 \
#     --top-p 0.9 \
#     --min-p 0.1 \
#     --repeat-penalty 1.1 \
#     --mlock \
#     --batch-size 16

openai_api_key = "NA"
openai_api_base = "http://localhost:8000/v1"
model_name = "qwen2.5-7b-instruct-q4_0.gguf"
model = ChatOpenAI(
    api_key = openai_api_key,
    base_url = openai_api_base,
    model_name = model_name
)

In [ ]:
import re 

def mongo_exec(ai_message):
    try:
        return f"The result of the query is:\n{eval(ai_message)}."
    except Exception as e:
        pattern = re.compile(r"\bcollection")
        if pattern.search(ai_message):
            return f"Extract only the query from {ai_message}."
        else:
            return "Invalid query. Please try again."

In [77]:
# Option 2: For extracting the number of errors.
# pipeline = [
#     {
#         "$match": { "src": "AMF" } 
#     },
#     {
#         "$count": "num_errors"  # $count requires a string field name
#     }
# ]

# result = list(collection.aggregate(pipeline))
# print(result) 
chain = prompt | model
chain.invoke(
    {
        "schema": collection.find_one({}),
        "history": [
            {"role": "user", "content": "Example:- Count the number of errors with source as UDR"},
            {"role": "assistant", "content": """collection.count_documents({"src": "AMF"})"""}
        ],
        "query": "Count the number of errors with destination as GNB"
    }
)

AIMessage(content='collection.count_documents({"dst": "GNB"})', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 483, 'total_tokens': 494, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2.5-7b-instruct-q4_0.gguf', 'system_fingerprint': 'b4970-c7b43ab6', 'id': 'chatcmpl-temRHzYH6lQBlEW7HTogxSremBUDWPgY', 'finish_reason': 'stop', 'logprobs': None}, id='run-fe5bfb75-6857-440c-b572-d9765902a41e-0', usage_metadata={'input_tokens': 483, 'output_tokens': 11, 'total_tokens': 494, 'input_token_details': {}, 'output_token_details': {}})

In [88]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser() | mongo_exec
chain.invoke(
    {
        "schema": collection.find_one({}),
        "history": [
            {"role": "user", "content": "Example:- Count the number of errors with source as UDR"},
            {"role": "assistant", "content": """collection.count_documents({"src": "AMF"})"""}
        ],
        "query": "Count the number of errors with destination as GNB"
    }
)

'The result of the query is: 2.'